In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Assigment4').getOrCreate()

In [2]:
# Load the new hotels file
base_df = spark.read.csv('Hotels_data_Changed.csv',inferSchema=True,header=True)

In [3]:
base_df.show(5)

+-----------+-------------------+-------------------+----+--------------+--------------+-------------+---------------+--------------------+-----------+-------+------------+-------------+-------+
|Snapshot ID|      Snapshot Date|       Checkin Date|Days|Original Price|Discount Price|Discount Code|Available Rooms|          Hotel Name|Hotel Stars|WeekDay|DiscountDiff| DiscountPerc|DayDiff|
+-----------+-------------------+-------------------+----+--------------+--------------+-------------+---------------+--------------------+-----------+-------+------------+-------------+-------+
|          1|2015-07-17 00:00:00|2015-08-12 00:00:00|   5|          1178|          1040|            1|              6|Best Western Plus...|          3|    Wed|         138| 11.714770798|     26|
|          1|2015-07-17 00:00:00|2015-08-19 00:00:00|   5|          1113|           982|            1|              8|Best Western Plus...|          3|    Wed|         131|11.7699910153|     33|
|          1|2015-07-17 0

In [4]:
from pyspark.sql.functions import col

# Get 150 Hotels that have the most rows in data
tophotels = base_df.groupBy("Hotel Name").count().sort(col("count").desc()).head(150)
tophotels

[Row(Hotel Name='Newark Liberty International Airport Marriott', count=5346),
 Row(Hotel Name='Hilton Garden Inn Times Square', count=4892),
 Row(Hotel Name='Residence Inn Newark Elizabeth Liberty International Airport', count=4314),
 Row(Hotel Name='Westin New York at Times Square', count=3792),
 Row(Hotel Name='Loews Regency New York Hotel', count=3617),
 Row(Hotel Name='Viceroy New York', count=3565),
 Row(Hotel Name='Four Seasons Hotel New York', count=3243),
 Row(Hotel Name='Langham Place New York Fifth Avenue', count=3203),
 Row(Hotel Name='The Carlyle A Rosewood Hotel', count=3078),
 Row(Hotel Name='DoubleTree by Hilton Metropolitan - New York City', count=2866),
 Row(Hotel Name='Magnuson Convention Center Hotel', count=2862),
 Row(Hotel Name='Hilton Garden Inn New York West 35th Street', count=2822),
 Row(Hotel Name='Hilton Garden Inn New York-Times Square Central', count=2772),
 Row(Hotel Name='Conrad New York', count=2677),
 Row(Hotel Name='Wyndham Garden Brooklyn Sunset Park

In [5]:
# convert the hotel name + count list to dataframe
top_hotel_names_df = spark.sparkContext.parallelize(tophotels).toDF(['hotel_name','COUNT'])

In [6]:
# Get 40 Checkin dates that have the most rows in data
topdates = base_df.groupBy("Checkin Date").count().sort(col("count").desc()).head(40)
topdates

[Row(Checkin Date=datetime.datetime(2015, 11, 11, 0, 0), count=2403),
 Row(Checkin Date=datetime.datetime(2015, 10, 28, 0, 0), count=2087),
 Row(Checkin Date=datetime.datetime(2015, 11, 6, 0, 0), count=2026),
 Row(Checkin Date=datetime.datetime(2015, 10, 14, 0, 0), count=2017),
 Row(Checkin Date=datetime.datetime(2015, 11, 4, 0, 0), count=2014),
 Row(Checkin Date=datetime.datetime(2015, 10, 21, 0, 0), count=1979),
 Row(Checkin Date=datetime.datetime(2015, 8, 19, 0, 0), count=1954),
 Row(Checkin Date=datetime.datetime(2015, 11, 5, 0, 0), count=1868),
 Row(Checkin Date=datetime.datetime(2015, 10, 29, 0, 0), count=1821),
 Row(Checkin Date=datetime.datetime(2015, 8, 12, 0, 0), count=1818),
 Row(Checkin Date=datetime.datetime(2015, 10, 22, 0, 0), count=1748),
 Row(Checkin Date=datetime.datetime(2015, 11, 12, 0, 0), count=1732),
 Row(Checkin Date=datetime.datetime(2015, 9, 10, 0, 0), count=1729),
 Row(Checkin Date=datetime.datetime(2015, 10, 30, 0, 0), count=1728),
 Row(Checkin Date=datetime

In [7]:
# convert the name + count list to dataframe
top_checkin_dates_df = spark.sparkContext.parallelize(topdates).toDF(['checkin_date','COUNT'])

In [8]:
only_hotel_names_df = top_hotel_names_df.drop("COUNT")
only_checkin_dates_df = top_checkin_dates_df.drop("COUNT")
joint_df = only_hotel_names_df.crossJoin(only_checkin_dates_df)

In [9]:
price_codes_df = spark.range(1,5).withColumnRenamed("id","price_code")
joint__with_price_codes_df = joint_df.crossJoin(price_codes_df)

In [10]:
joint__with_price_codes_df.count()

24000

In [11]:
dummy_prices_df = spark.range(-1,0).withColumnRenamed("id","price")
complete_df = joint__with_price_codes_df.crossJoin(dummy_prices_df)

In [12]:
complete_df.show()

+--------------------+-------------------+----------+-----+
|          hotel_name|       checkin_date|price_code|price|
+--------------------+-------------------+----------+-----+
|Newark Liberty In...|2015-11-11 00:00:00|         1|   -1|
|Newark Liberty In...|2015-11-11 00:00:00|         2|   -1|
|Newark Liberty In...|2015-11-11 00:00:00|         3|   -1|
|Newark Liberty In...|2015-11-11 00:00:00|         4|   -1|
|Newark Liberty In...|2015-10-28 00:00:00|         1|   -1|
|Newark Liberty In...|2015-10-28 00:00:00|         2|   -1|
|Newark Liberty In...|2015-10-28 00:00:00|         3|   -1|
|Newark Liberty In...|2015-10-28 00:00:00|         4|   -1|
|Newark Liberty In...|2015-11-06 00:00:00|         1|   -1|
|Newark Liberty In...|2015-11-06 00:00:00|         2|   -1|
|Newark Liberty In...|2015-11-06 00:00:00|         3|   -1|
|Newark Liberty In...|2015-11-06 00:00:00|         4|   -1|
|Newark Liberty In...|2015-10-14 00:00:00|         1|   -1|
|Newark Liberty In...|2015-10-14 00:00:0

In [13]:
for row in complete_df.collect():
    base_df.where(row.)
data_list.show()

NameError: name 'data_list' is not defined